In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"

#initialize pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

#create index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name = index_name,
        dimension = 384,   #dimension of dense vector
        metric = "dotproduct",   #sparse valuse supported only for dot product
        spec = ServerlessSpec(cloud ="aws",region=  "us-east-1"),
        )

In [4]:
index = pc.Index(index_name)
index

In [7]:
import torch
torch.cuda.is_available()

True

In [ ]:
## vector embedding and sparse metrix

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLm-L6-v2")
embeddings

In [10]:
from pinecone_text.sparse import BM25Encoder   #uses TFIDF by default

bm25_encoder = BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt to /home/jivan-
[nltk_data]     acharya/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/jivan-
[nltk_data]     acharya/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
sentences =[
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans"
]

##tfidf values on these sentences
bm25_encoder.fit(sentences)

## store to json
bm25_encoder.dump("bm25_values.json")

#load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 3943.25it/s]


In [ ]:
retriever = PineconeHybridSearchRetriever(embeddings = embeddings, sparse_encoder = bm25_encoder, index = index)

In [ ]:
retriever.add_texts(sentences)

In [ ]:
retriever.invoke("what city did i visit last")